# `nanoGPT`: GPT-2 Small (125M Params)

## Install / Setup

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

python3 -c 'import ngpt; print(ngpt.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has ngpt installed. Nothing to do."
else
    echo "Does not have ngpt installed. Installing..."
    git clone 'https://github.com/saforem2/nanoGPT'
    python3 nanoGPT/data/shakespeare_char/prepare.py
    python3 -m pip install -e nanoGPT -vvv
fi

/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py
Has ngpt installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from enrich import get_logger
log = get_logger('jupyter')
log.info(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2023-11-29 19:41:53][INFO][3434626787.py:7] - /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config
from ngpt.trainer import Trainer

os.environ['MASTER_PORT'] = '4235'
rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=owt',              # open web text
        'model=gpt2',            # gpt2 arch.
        'optimizer=gpt2',
        'train=gpt2',
        'train.init_from=gpt2',  # init from GPT2
        'train.max_iters=1000',
        'train.dtype=bfloat16',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu24
  Local device: mlx5_0
--------------------------------------------------------------------------


Failed to download font: Source Sans Pro, skipping!
Failed to download font: Titillium WebRoboto Condensed, skipping!
[2023-11-29 19:42:01][WARNING][configs.py:297] - No meta.pkl found, assuming GPT-2 encodings...
[2023-11-29 19:42:01][INFO][configs.py:263] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-29 19:42:01][INFO][configs.py:398] - Tokens per iteration: 12,288
[2023-11-29 19:42:01][INFO][configs.py:430] - Using <torch.amp.autocast_mode.autocast object at 0x7f6e4c363550>
[2023-11-29 19:42:01][INFO][trainer.py:187] - Initializing from OpenAI GPT-2 Weights: gpt2
[2023-11-29 19:42:02][INFO][model.py:225] - loading weights from pretrained gpt: gpt2
[2023-11-29 19:42:02][INFO][model.py:234] - forcing vocab_size=50257, block_size=1024, bias=True
[2023-11-29 19:42:02][INFO][model.py:240] - overriding dropout rate to 0.0
[2023-11-29 19:42:05][INFO][model.py:160] - number of parameters: 123.65M


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

[2023-11-29 19:42:10][INFO][model.py:290] - num decayed parameter tensors: 50, with 124,318,464 parameters
[2023-11-29 19:42:10][INFO][model.py:291] - num non-decayed parameter tensors: 98, with 121,344 parameters
[2023-11-29 19:42:10][INFO][model.py:297] - using fused AdamW: True


In [4]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-29 19:42:25][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-29 19:42:25][INFO][1657463709.py:4] - ['response']:

What is a supercomputer? When did you first learn it?

I used to work in an Apple Computer. It was called the "Elgin" Computer. It was the first computer that I had seen on TV. I went to college in 1983. I was at Arizona State University and I studied computer science. I later joined a computer science program at MIT. But my first computer was the Intel Core. It was from 1986. I went to MIT where I got my PhD and was at the lab. When I did graduate school in 1987, I went to Stanford where I made a few jobs.

Did you ever think of programming as the "new media?" Did you ever think of learning about computers as the "new medium?"

The world was changing. I started to think about the Internet as a new medium. I thought of computers as different from television and movies. My research led me to the Computer World. Where did you get that idea? It was a computer 

## Train Model

Legend:

<div style="text-align:left;">

|  **NAME**  |     **DESCRIPTION**          |
|:----------:|:----------------------------:|
|   `step`   | Current training step        |
|   `loss`   | Loss value                   |
|   `dt`     | Time per step (in **ms**)    |
|   `sps`    | Samples per second           |
|   `mtps`   | (million) Tokens per sec     |
|   `mfu`    | Model Flops Utilization*     |

*in units of A100 `bfloat16` peak FLOPS

</div>

In [5]:
trainer.train(train_iters=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

[2023-11-29 19:43:53][INFO][trainer.py:516] - step=100 loss=3.119 dt=254.990 sps=3.922 mtps=0.048 mfu=13.208 train_loss=3.126 val_loss=3.102
[2023-11-29 19:44:20][INFO][trainer.py:516] - step=200 loss=2.961 dt=228.112 sps=4.384 mtps=0.054 mfu=13.364 train_loss=3.126 val_loss=3.102
[2023-11-29 19:44:46][INFO][trainer.py:516] - step=300 loss=3.022 dt=241.652 sps=4.138 mtps=0.051 mfu=13.421 train_loss=3.126 val_loss=3.102
[2023-11-29 19:45:12][INFO][trainer.py:516] - step=400 loss=3.002 dt=231.064 sps=4.328 mtps=0.053 mfu=13.537 train_loss=3.126 val_loss=3.102
[2023-11-29 19:45:37][INFO][trainer.py:516] - step=500 loss=3.004 dt=253.217 sps=3.949 mtps=0.049 mfu=13.513 train_loss=3.126 val_loss=3.102
[2023-11-29 19:46:03][INFO][trainer.py:516] - step=600 loss=3.071 dt=239.040 sps=4.183 mtps=0.051 mfu=13.571 train_loss=3.126 val_loss=3.102
[2023-11-29 19:46:30][INFO][trainer.py:516] - step=700 loss=3.118 dt=221.668 sps=4.511 mtps=0.055 mfu=13.733 train_loss=3.126 val_loss=3.102
[2023-11-29 1

## Evaluate Model

In [7]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-29 19:49:09][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-29 19:49:09][INFO][1657463709.py:4] - ['response']:

What is a supercomputer?

Researchers at MIT and EPFL and the University of Southern California published a paper on July 17 in IEEE Translational Computer Graphics. The paper, titled "Processes for Supercomputers," "is particularly interesting because it shows how computer graphics can be used with supercomputers."

Supercomputers solve many of the processing needs of large graphical full-screen displays, and there is always room to improve one's graphics. Several supercomputers have been used to prototype computer designs, and "this is one of the first supercomputers to try and develop a very high-level, powerful GPU," says Daniel Pape. Pape and his team "are working on a GPU implementation" of processes, he says.

The paper explains how supercomputers program computing using the language of higher-order models, but there is "no formal description" for wh